In [28]:
from IPython.display import clear_output,display_svg,HTML,display
from ipywidgets.widgets import IntSlider,FloatSlider,interact
import numpy as np
from random import randint,random
from graphviz import Digraph
from pprint import pprint

def full_graph(cities,dist_table):
    graph = Digraph(engine='circo')
    for i in range(0,cities):
        graph.node(str(i),shape='point')
    for edge in dist_table:
        t = list(edge)
        graph.edge(str(t[0]),str(t[1]),dir='none',color='gray85',)
    return graph

def path_graph(cities,dist_table,path):
    graph = Digraph(engine='circo')
    for i in range(0,cities):
        graph.node(str(i))
    for edge in dist_table:
        t = list(edge)
        if edge in path:
            graph.edge(str(t[0]),str(t[1]),dir='none',color='red',label=str(dist_table[edge][0]),labelfontsize='15')
        else:
            graph.edge(str(t[0]),str(t[1]),dir='none',color='gray85')
    return graph



In [29]:
class Ant():
    def __init__(self):
        self.current_city = 0
        self.path = []
        self.travel_length = 0
        self.paths = []

    def __repr__(self):
        return 'Длина пути {}, Текущий город {}, Посещенные города {},Дороги {}'.format(self.travel_length,self.current_city,self.path,self.paths)

In [30]:
def generate_cities(count_of_cities,max_dist):
    path_map = {}
    for i in range(count_of_cities-1):
        for j in range(i,count_of_cities):
            if not i == j: 
                path_map[frozenset([i,j])] = [randint(1,max_dist),1.0/count_of_cities]
    return path_map

def init_ants(count_of_ants,count_of_cities):
    ants = []
    current_city = 0
    for i in range(count_of_ants):
        if current_city>=count_of_cities:
            current_city = 0
        ant = Ant()
        ant.current_city = current_city
        ant.path.append(current_city)
        ants.append(ant)
        current_city += 1
    return ants

def restart_ants(ants,count_of_ants,count_of_cities):
    best = min(ants,key = lambda a: a.travel_length)
    ants.clear()
    current_city = 0
    for i in range(count_of_ants):
        if current_city>=count_of_cities:
            current_city = 0
        ant = Ant()
        ant.current_city = current_city
        ant.path.append(current_city)
        ants.append(ant)
        current_city += 1
    return best

def ant_product(path_map,alpha,beta,a,b):
    return (path_map[frozenset([a,b])][1]**alpha) * (path_map[frozenset([a,b])][0]**beta)

def select_next_city(ant,count_of_cities,path_map,alpha,beta):
    denom = 0.0
    a = ant.current_city
    avalible_paths =[]
    for i in range(count_of_cities):
        if i not in ant.path:
            denom += ant_product(path_map,alpha,beta,a,i)
            avalible_paths.append(
                [a,i,path_map[frozenset([a,i])]]
            )
    if len(avalible_paths) == 0:
        return None 
    
    flag = True
    result_city = None
    while(flag):
        for path in avalible_paths:
            p = 0.0
            p = ant_product(path_map,alpha,beta,path[0],path[1])
            bypass = random()
            if bypass<p:
                result_city = path[1]
                flag = False
                break
    return result_city

def simulate_ants(ants,count_of_cities,path_map,alpha,beta):
    moving = 0
    for ant in ants:
        if len(ant.path) < count_of_cities:
            next_city = select_next_city(ant,count_of_cities,path_map,alpha,beta)
            if next_city == None:
                next
            ant.path.append(next_city)
            ant.paths.append(frozenset([ant.current_city,next_city]))
            ant.travel_length += path_map[
                frozenset([
                    ant.current_city,
                    next_city
                ])
            ][0]
            ant.current_city = next_city
            moving += 1
    return moving

def update_trails(ants,count_of_cities,path_map,rho,qval):
    for path in path_map:
        if path_map[path][1] < 0.0:
            path_map[path][1] = 1.0 / count_of_cities
        else:
            path_map[path][1] *= rho
    for ant in ants:
        for path in ant.paths:
            path_map[path][1] += (qval/ant.travel_length) 
    for path in path_map:
        path_map[path][1] *= rho  

def ant_return_to_start(ants,path_map):
    for ant in ants:
        ant.paths.append(frozenset([ant.path[0],ant.path[-1]]))
        ant.travel_length += path_map[
            frozenset([ant.path[0],ant.path[-1]])][0]

In [31]:
@interact(
    cities = IntSlider(min=3,max=30),
    max_dist = IntSlider(min=2,max=50)
)

def city_generator(cities,max_dist):
    path_map = generate_cities(cities,max_dist)
    display_svg(
        full_graph(cities,path_map)
    )
    @interact(
        ant_count = IntSlider(min=5,max=100),
        step_count = IntSlider(min=20,max=50),
        alpha = FloatSlider(min = 0.1, max=1.0),
        beta = FloatSlider(min = 0.1, max=1.0),
        rho = FloatSlider(min = 0.1, max=1.0),
        qval = FloatSlider(min = 0.1, max=1.0)
    )
    def ants_alg(ant_count,step_count,alpha,beta,rho,qval):
        best_ants = []
        ants = init_ants(ant_count,cities)
        for step in range(step_count):
            while(simulate_ants(ants,cities,path_map,alpha,beta) != 0):
                pass
            update_trails(ants,cities,path_map,rho,qval)
            ant_return_to_start(ants,path_map)
            best_ants.append(
                restart_ants(ants,ant_count,cities)
            )
        pprint(best_ants)
        display_svg(
            path_graph(cities,path_map,best_ants[-1].paths)
        )

interactive(children=(IntSlider(value=3, description='cities', max=30, min=3), IntSlider(value=2, description=…